In [1]:
import nltk
import sklearn_crfsuite

from copy import deepcopy
from collections import defaultdict

from ner_evaluation.ner_eval import collect_named_entities
from ner_evaluation.ner_eval import compute_metrics

## Train a CRF on the CoNLL 2002 NER Spanish data

In [2]:
nltk.corpus.conll2002.fileids()
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for token, postag, label in sent]


def sent2tokens(sent):
    return [token for token, postag, label in sent]

## Feature Extraction

In [4]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 1.12 s, sys: 98.2 ms, total: 1.22 s
Wall time: 1.22 s


## Training

In [5]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 34.1 s, sys: 197 ms, total: 34.3 s
Wall time: 34.4 s


## Performance per label type per token

In [6]:
y_pred = crf.predict(X_test)
labels = list(crf.classes_)
labels.remove('O') # remove 'O' label from evaluation
sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0])) # group B and I results
print(sklearn_crfsuite.metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

             precision    recall  f1-score   support

      B-LOC      0.810     0.784     0.797      1084
      I-LOC      0.690     0.637     0.662       325
     B-MISC      0.731     0.569     0.640       339
     I-MISC      0.699     0.589     0.639       557
      B-ORG      0.807     0.832     0.820      1400
      I-ORG      0.852     0.786     0.818      1104
      B-PER      0.850     0.884     0.867       735
      I-PER      0.893     0.943     0.917       634

avg / total      0.809     0.787     0.796      6178



## Performance over full named-entity

In [7]:
test_sents_labels = []
for sentence in test_sents:
    sentence = [token[2] for token in sentence]
    test_sents_labels.append(sentence)

In [8]:
index = 2
true = collect_named_entities(test_sents_labels[index])
pred = collect_named_entities(y_pred[index])

In [9]:
true

[Entity(e_type='MISC', start_offset=12, end_offset=12),
 Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='PER', start_offset=37, end_offset=39),
 Entity(e_type='ORG', start_offset=45, end_offset=46)]

In [10]:
pred

[Entity(e_type='MISC', start_offset=12, end_offset=12),
 Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='PER', start_offset=37, end_offset=39),
 Entity(e_type='LOC', start_offset=45, end_offset=46)]

In [11]:
index = 2
true = collect_named_entities(test_sents_labels[index])
pred = collect_named_entities(y_pred[index])

In [12]:
true

[Entity(e_type='MISC', start_offset=12, end_offset=12),
 Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='PER', start_offset=37, end_offset=39),
 Entity(e_type='ORG', start_offset=45, end_offset=46)]

In [13]:
pred

[Entity(e_type='MISC', start_offset=12, end_offset=12),
 Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='PER', start_offset=37, end_offset=39),
 Entity(e_type='LOC', start_offset=45, end_offset=46)]

In [17]:
compute_metrics(true, pred)

({'ent_type': {'actual': 4,
   'correct': 3,
   'incorrect': 1,
   'missed': 0,
   'partial': 0,
   'possible': 4,
   'precision': 0.75,
   'recall': 0.75,
   'spurius': 0},
  'strict': {'actual': 4,
   'correct': 3,
   'incorrect': 1,
   'missed': 0,
   'partial': 0,
   'possible': 4,
   'precision': 0.75,
   'recall': 0.75,
   'spurius': 0}},
 {'LOC': {'ent_type': {'correct': 1,
    'incorrect': 1,
    'missed': 0,
    'partial': 0,
    'spurius': 0},
   'strict': {'correct': 1,
    'incorrect': 1,
    'missed': 0,
    'partial': 0,
    'spurius': 0}},
  'MISC': {'ent_type': {'correct': 1,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 0},
   'strict': {'correct': 1,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 0}},
  'ORG': {'ent_type': {'correct': 0,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 0},
   'strict': {'correct': 0,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 0}},
  'PER': {

In [18]:
to_test = [2,4,12,14]

In [19]:
index = 2
true_named_entities_type = defaultdict(list)
pred_named_entities_type = defaultdict(list)

for true in collect_named_entities(test_sents_labels[index]):
    true_named_entities_type[true.e_type].append(true)

for pred in collect_named_entities(y_pred[index]):
    pred_named_entities_type[pred.e_type].append(pred)

In [20]:
true_named_entities_type

defaultdict(list,
            {'LOC': [Entity(e_type='LOC', start_offset=15, end_offset=15)],
             'MISC': [Entity(e_type='MISC', start_offset=12, end_offset=12)],
             'ORG': [Entity(e_type='ORG', start_offset=45, end_offset=46)],
             'PER': [Entity(e_type='PER', start_offset=37, end_offset=39)]})

In [21]:
pred_named_entities_type

defaultdict(list,
            {'LOC': [Entity(e_type='LOC', start_offset=15, end_offset=15),
              Entity(e_type='LOC', start_offset=45, end_offset=46)],
             'MISC': [Entity(e_type='MISC', start_offset=12, end_offset=12)],
             'PER': [Entity(e_type='PER', start_offset=37, end_offset=39)]})

In [22]:
true_named_entities_type['LOC']

[Entity(e_type='LOC', start_offset=15, end_offset=15)]

In [23]:
pred_named_entities_type['LOC']

[Entity(e_type='LOC', start_offset=15, end_offset=15),
 Entity(e_type='LOC', start_offset=45, end_offset=46)]

In [24]:
compute_metrics(true_named_entities_type['LOC'], pred_named_entities_type['LOC'])

({'ent_type': {'actual': 2,
   'correct': 1,
   'incorrect': 0,
   'missed': 0,
   'partial': 0,
   'possible': 1,
   'precision': 0.5,
   'recall': 1.0,
   'spurius': 1},
  'strict': {'actual': 2,
   'correct': 1,
   'incorrect': 0,
   'missed': 0,
   'partial': 0,
   'possible': 1,
   'precision': 0.5,
   'recall': 1.0,
   'spurius': 1}},
 {'LOC': {'ent_type': {'correct': 1,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 1},
   'strict': {'correct': 1,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 1}},
  'MISC': {'ent_type': {'correct': 0,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 0},
   'strict': {'correct': 0,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 0}},
  'ORG': {'ent_type': {'correct': 0,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 0},
   'strict': {'correct': 0,
    'incorrect': 0,
    'missed': 0,
    'partial': 0,
    'spurius': 0}},
  'PER': {'ent

## results over all messages

In [25]:
metrics_results = {'correct': 0, 'incorrect': 0, 'partial': 0,
                   'missed': 0, 'spurius': 0, 'possible': 0, 'actual': 0}

# overall results
results = {'strict': deepcopy(metrics_results),
           'ent_type': deepcopy(metrics_results)
           }

# results aggregated by entity type
evaluation_agg_entities_type = {e: deepcopy(results) for e in ['LOC','PER','ORG','MISC']}

for true_ents, pred_ents in zip(test_sents_labels, y_pred):    
    # compute results for one message
    tmp_results, tmp_agg_results = compute_metrics(collect_named_entities(true_ents),collect_named_entities(pred_ents))
    
    # aggregate overall results
    for eval_schema in results.keys():
        for metric in metrics_results.keys():
            results[eval_schema][metric] += tmp_results[eval_schema][metric]

    # aggregate results by entity type
    for e_type in ['LOC','PER','ORG','MISC']:
        for eval_schema in tmp_agg_results[e_type]:
            for metric in tmp_agg_results[e_type][eval_schema]:
                evaluation_agg_entities_type[e_type][eval_schema][metric] += tmp_agg_results[e_type][eval_schema][metric]


In [26]:
results

{'ent_type': {'actual': 3518,
  'correct': 2909,
  'incorrect': 564,
  'missed': 111,
  'partial': 0,
  'possible': 3584,
  'spurius': 45},
 'strict': {'actual': 3518,
  'correct': 2779,
  'incorrect': 694,
  'missed': 111,
  'partial': 0,
  'possible': 3584,
  'spurius': 45}}

In [27]:
evaluation_agg_entities_type

{'LOC': {'ent_type': {'actual': 0,
   'correct': 861,
   'incorrect': 180,
   'missed': 32,
   'partial': 0,
   'possible': 0,
   'spurius': 5},
  'strict': {'actual': 0,
   'correct': 840,
   'incorrect': 201,
   'missed': 32,
   'partial': 0,
   'possible': 0,
   'spurius': 5}},
 'MISC': {'ent_type': {'actual': 0,
   'correct': 211,
   'incorrect': 46,
   'missed': 33,
   'partial': 0,
   'possible': 0,
   'spurius': 7},
  'strict': {'actual': 0,
   'correct': 173,
   'incorrect': 84,
   'missed': 33,
   'partial': 0,
   'possible': 0,
   'spurius': 7}},
 'ORG': {'ent_type': {'actual': 0,
   'correct': 1181,
   'incorrect': 231,
   'missed': 34,
   'partial': 0,
   'possible': 0,
   'spurius': 31},
  'strict': {'actual': 0,
   'correct': 1120,
   'incorrect': 292,
   'missed': 34,
   'partial': 0,
   'possible': 0,
   'spurius': 31}},
 'PER': {'ent_type': {'actual': 0,
   'correct': 656,
   'incorrect': 107,
   'missed': 12,
   'partial': 0,
   'possible': 0,
   'spurius': 2},
  'str